In [7]:
import os
import glob
import pandas as pd

In [8]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/smpd3_dataset/20171016_gRNA1_Pax7_Sox9/sec/sox9counts'

In [9]:
# Add experiment date here to apply to dataframe
expt_date = '20171016'
treatment = 'SMPD3gRNA1'
result_type = 'Sox9Counts'

path = os.path.abspath('')+'/sox9csv/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Date'] = expt_date
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Parse Image name
    df['Embryo'], df['Somites'], df['Section'] = zip(*df['Image'].map(lambda x: x.split('_')))

    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,Slice,Count,Total Area,Average Size,%Area,Mean,IntDen,Date,Image,Embryo,Somites,Section
0,CntlSide,14,508.127,36.295,7.690,255,9255.179,20171016,Emb9_9ss_sec3,Emb9,9ss,sec3
1,ExptSide,13,705.535,54.272,6.488,255,13839.342,20171016,Emb9_9ss_sec3,Emb9,9ss,sec3
0,CntlSide,15,586.864,39.124,6.258,255,9976.684,20171016,Emb9_9ss_sec2,Emb9,9ss,sec2
1,ExptSide,12,485.146,40.429,4.206,255,10309.343,20171016,Emb9_9ss_sec2,Emb9,9ss,sec2
0,CntlSide,32,1510.830,47.213,10.472,255,12039.429,20171016,Emb9_9ss_sec1,Emb9,9ss,sec1


In [10]:
# Mean counts across sections
mean_sections = pd.DataFrame((full_df.groupby(['Embryo', 'Slice'])['Count']).mean())
mean_sections
# Extract means for Cntl and Expt counts
sox9_cntl = mean_sections.xs('CntlSide', level='Slice')['Count'] 
sox9_expt = mean_sections.xs('ExptSide', level='Slice')['Count']

# Generate ratios as Expt/Cntl
sox9_ratios = pd.DataFrame(sox9_expt / sox9_cntl)
sox9_ratios.columns = ['Expt/Cntl Sox9']

In [11]:
# Combine processed values into single dataframe and output as csv file
sox9_cntl = pd.DataFrame(sox9_cntl)
sox9_cntl.columns = ['Cntl Sox9']
sox9_expt = pd.DataFrame(sox9_expt)
sox9_expt.columns = ['Expt Sox9']

results = pd.concat([sox9_cntl, sox9_expt, sox9_ratios], axis=1, sort=True)
results['ID'] = expt_date + '_' + results.index
results.to_csv(expt_date + '_' + treatment + '_' + result_type + '_results.csv')